In [2]:
import numpy as np
from pyomo.environ import *
import pandas as pd

In [15]:
def load_distance_time_matrix(path, N):
    data= pd.read_csv(path)
    distance = np.zeros((N,N))
    time = np.zeros((N,N))
    
    for i in range(len(data)):
        origen = int(data.iloc[i, 0])
        destino = int(data.iloc[i, 1])
        distance[origen-1, destino-1] = float(data.iloc[i, 2])
        time[origen-1, destino-1] = float(data.iloc[i, 3])
    return distance, time

def load_vehicles(path):
    data= pd.read_csv(path)
    N = len(data)
    vehicles = {}
    for i in range(N):
        id = int(data.iloc[i, 0])
        capacity = int(data.iloc[i, 1])
        ran = float(data.iloc[i, 2])
        vehicles[id] = (capacity, ran)
    
    return vehicles

def load_demand(clientsPath):
    data= pd.read_csv(clientsPath)
    N = len(data)
    demand_dic = {}
    for i in range(N):
        id = int(data.iloc[i, 1])
        demand = float(data.iloc[i, 2])
        demand_dic[id] = demand
    
    return demand_dic


def load_capacity(depositPath):
    data= pd.read_csv(depositPath)
    N = len(data)
    capacity_dic = {}
    for i in range(N):
        id = int(data.iloc[i, 1])
        capacity = float(data.iloc[i, 4])
        capacity_dic[id] = capacity
    
    return capacity_dic

def load_distance_time_dic(path):
    data= pd.read_csv(path)
    distance = {}
    time = {}
    
    for i in range(len(data)):
        origen = int(data.iloc[i, 0])
        destino = int(data.iloc[i, 1])
        distance[origen, destino] = float(data.iloc[i, 2])
        time[origen, destino] = float(data.iloc[i, 3])
    
    return distance, time

def load_coordinates(depotsPath, clientsPath):
    coord = {}
    depot= pd.read_csv(depotsPath)
    client= pd.read_csv(clientsPath)
    
    for i in range(len(depot)):
        id = int(depot.iloc[i, 1])
        lat = float(depot.iloc[i, 3])
        long = float(depot.iloc[i, 2])
        coord[id] = [lat, long]

    for j in range(len(client)):
        id = int(client.iloc[j, 1])
        lat = float(client.iloc[j, 4])
        long = float(client.iloc[j, 3])
        coord[id] = [lat, long]
    
    return coord
        

In [17]:
#Cargar datos
distancia,_ = load_distance_time_dic('../Datos/Caso2/caso2.csv')
vehiculos = load_vehicles('../Datos/Caso2/vehicles.csv')
demanda = load_demand('..\Datos\Caso2\clients.csv')
oferta = load_capacity('..\Datos\Caso2\depots.csv')
coord = load_coordinates('..\Datos\Caso2\depots.csv', '..\Datos\Caso2\clients.csv')

print(f"Distancia: {distancia}")
print(f"Vehiculos: {vehiculos}")
print(f"Oferta: {oferta}")
print(f"Demanda: {demanda}")


Distancia: {(1, 2): 33061.2, (1, 3): 10160.7, (1, 4): 6452.9, (1, 5): 21415.6, (1, 6): 16924.1, (1, 7): 20126.9, (1, 8): 11361.2, (1, 9): 4329.9, (1, 10): 31289.5, (1, 11): 13659.0, (1, 12): 26223.2, (1, 13): 22414.9, (1, 14): 25897.0, (1, 15): 4225.5, (1, 16): 22338.8, (1, 17): 12064.8, (1, 18): 18088.6, (1, 19): 19570.0, (1, 20): 10485.2, (1, 21): 18553.2, (2, 1): 32946.7, (2, 3): 34180.2, (2, 4): 29493.6, (2, 5): 13916.9, (2, 6): 18526.9, (2, 7): 14575.1, (2, 8): 23069.6, (2, 9): 31601.6, (2, 10): 2571.4, (2, 11): 19234.4, (2, 12): 7378.4, (2, 13): 20179.2, (2, 14): 15095.2, (2, 15): 32532.2, (2, 16): 21842.3, (2, 17): 30803.4, (2, 18): 23599.4, (2, 19): 25095.8, (2, 20): 29219.4, (2, 21): 16821.9, (3, 1): 14330.3, (3, 2): 38912.1, (3, 4): 15346.8, (3, 5): 32569.6, (3, 6): 28926.9, (3, 7): 26663.9, (3, 8): 23532.5, (3, 9): 18517.0, (3, 10): 37140.4, (3, 11): 24943.3, (3, 12): 32074.1, (3, 13): 35803.1, (3, 14): 34588.3, (3, 15): 18412.6, (3, 16): 35727.1, (3, 17): 25617.5, (3, 18): 

In [18]:
# Modelo
modelP2 = ConcreteModel()

# Conjuntos
modelP2.N = Set(initialize=set(oferta.keys()).union(set(demanda.keys())))  # Todos los nodos
modelP2.C = Set(initialize=demanda.keys())  # Clientes
modelP2.D = Set(initialize=oferta.keys())   # Centros de distribución
modelP2.V = Set(initialize=vehiculos.keys())  # Vehículos

# Variables
modelP2.x = Var(modelP2.N, modelP2.N, modelP2.V, domain=Binary)  # Ruta de i a j por v
modelP2.u = Var(modelP2.N, modelP2.V, domain=NonNegativeReals)  # Carga acumulada en i por v
modelP2.start = Var(modelP2.D, modelP2.V, domain=Binary)  # 1 si vehículo v parte desde depósito d

# Parámetros
modelP2.dist = Param(modelP2.N, modelP2.N, initialize=distancia, default=0)
modelP2.demand = Param(modelP2.C, initialize=demanda)
modelP2.offer = Param(modelP2.D, initialize=oferta)
modelP2.cap = Param(modelP2.V, initialize={v: vehiculos[v][0] for v in vehiculos})
modelP2.range = Param(modelP2.V, initialize={v: vehiculos[v][1] for v in vehiculos})

# Función objetivo: minimizar distancia total recorrida
def obj_rule(m):
    return sum(m.dist[i, j] * m.x[i, j, v] for i in m.N for j in m.N if i != j for v in m.V)
modelP2.obj = Objective(rule=obj_rule, sense=minimize)

# -------------------------
# Restricciones
# -------------------------

# Cada cliente debe ser visitado exactamente una vez
modelP2.visit_once = ConstraintList()
for c in modelP2.C:
    modelP2.visit_once.add(
        sum(modelP2.x[i, c, v] for i in modelP2.N if i != c for v in modelP2.V) == 1
    )

# Conservación de flujo por vehículo para todos los nodos (excepto origen/destino)
modelP2.flow = ConstraintList()
for v in modelP2.V:
    for n in modelP2.N:
        modelP2.flow.add(
            sum(modelP2.x[i, n, v] for i in modelP2.N if i != n) ==
            sum(modelP2.x[n, j, v] for j in modelP2.N if j != n)
        )

# Cada vehículo parte de solo un depósito
modelP2.one_depot_per_vehicle = ConstraintList()
for v in modelP2.V:
    modelP2.one_depot_per_vehicle.add(
        sum(modelP2.start[d, v] for d in modelP2.D) == 1
    )

# Si vehículo v parte de depósito d, entonces debe salir desde allí
modelP2.start_link = ConstraintList()
for v in modelP2.V:
    for d in modelP2.D:
        modelP2.start_link.add(
            sum(modelP2.x[d, j, v] for j in modelP2.N if j != d) == modelP2.start[d, v]
        )

# Carga acumulada: capacidad máxima
modelP2.capacity_limit = ConstraintList()
for v in modelP2.V:
    for i in modelP2.N:
        modelP2.capacity_limit.add(modelP2.u[i, v] <= modelP2.cap[v])

# Carga acumulada: balance de carga si se recorre (subtour elimination, tipo MTZ)
modelP2.subtour_elimination = ConstraintList()
for v in modelP2.V:
    for i in modelP2.C:
        for j in modelP2.C:
            if i != j:
                modelP2.subtour_elimination.add(
                    modelP2.u[i, v] + modelP2.demand[j] - modelP2.u[j, v] <= modelP2.cap[v] * (1 - modelP2.x[i, j, v])
                )

# Carga inicial si parte desde un depósito
modelP2.initial_load = ConstraintList()
for v in modelP2.V:
    for d in modelP2.D:
        modelP2.initial_load.add(
            modelP2.u[d, v] == modelP2.start[d, v]
        )

# Total entregado desde cada depósito no excede su oferta
modelP2.offer_limit = ConstraintList()
for d in modelP2.D:
    modelP2.offer_limit.add(
        sum(modelP2.demand[j] * modelP2.x[d, j, v] for j in modelP2.C for v in modelP2.V if j != d) <= modelP2.offer[d]
    )

# Rango máximo del vehículo
modelP2.range_limit = ConstraintList()
for v in modelP2.V:
    modelP2.range_limit.add(
        sum(modelP2.dist[i, j] * modelP2.x[i, j, v] for i in modelP2.N for j in modelP2.N if i != j) <= modelP2.range[v]
    )

# No permitir ciclos triviales
modelP2.no_loops = ConstraintList()
for v in modelP2.V:
    for i in modelP2.N:
        modelP2.no_loops.add(modelP2.x[i, i, v] == 0)
        

# Resolver el modelo
opt = SolverFactory('glpk')
results = opt.solve(modelP2, tee=True)

# Resultados
print("Valor de la función objetivo:", modelP2.obj())
# Mostrar capacidad utilizada por ruta
routes = {}
for v in modelP2.V:
    print(f"Vehículo {v}:")
    routes[v] = []
    for i in modelP2.N:
        for j in modelP2.N:
            if i != j and modelP2.x[i, j, v].value > 0.5:  # Solo si la ruta es tomada
                origen = coord[i]
                destino = coord[j]
                
                if origen not in routes[v]:
                    routes[v].append(origen)
                if destino not in routes[v]:
                    routes[v].append(destino)
                
                if i in modelP2.C:
                    capacidad_utilizada = modelP2.demand[i] * modelP2.x[i, j, v].value
                    print(f"  Ruta {i} -> {j}: Carga transportada = {capacidad_utilizada}")
                else:
                    print(f"  Ruta {i} -> {j} (desde depósito)")


(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo
GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\aleja\AppData\Local\Temp\tmpl6n9t050.glpk.raw --wglp C:\Users\aleja\AppData\Local\Temp\tmpzfq1y4gc.glpk.glp
 --cpxlp C:\Users\aleja\AppData\Local\Temp\tmpmslyj_d4.pyomo.lp
Reading problem data from 'C:\Users\aleja\AppData\Local\Temp\tmpmslyj_d4.pyomo.lp'...
C:\Users\aleja\AppData\Local\Temp\tmpmslyj_d4.pyomo.lp:20899: warning: lower bound of variable 'x3582' redefined
C:\Users\aleja\AppData\Local\Temp\tmpmslyj_d4.pyomo.lp:20899: warning: upper bound of variable 'x3582' redefined
987 rows, 2844 columns, 12564 non-zeros
2718 integer variables, all of which are binary
23617 lines were read
Writing problem data to 'C:\Users\aleja\AppData\Local\Temp\tmpzfq1y4gc.glpk.glp'...
19635 lines were written
GLPK Integer Optimizer 5.0
987 rows, 2844 columns, 12564 non-zeros
2718 integer variables, all of which are binary
Preprocess

ValueError: No value for uninitialized NumericValue object x[1,2,1]